<a href="https://colab.research.google.com/github/cacress/CSC442/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import pandas
import pandas as pd

# Read in the file and print out the DataFrame
hulu_data = pd.read_csv("https://raw.githubusercontent.com/cacress/CSC442/refs/heads/main/datasets/titles_hulu.csv")
netflix_data = pd.read_csv("https://raw.githubusercontent.com/cacress/CSC442/refs/heads/main/datasets/titles_netflix.csv")

In [2]:
hulu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2398 non-null   object 
 1   title                 2398 non-null   object 
 2   type                  2398 non-null   object 
 3   description           2385 non-null   object 
 4   release_year          2398 non-null   int64  
 5   age_certification     1713 non-null   object 
 6   runtime               2398 non-null   int64  
 7   genres                2398 non-null   object 
 8   production_countries  2398 non-null   object 
 9   seasons               1330 non-null   float64
 10  imdb_id               2263 non-null   object 
 11  imdb_score            2232 non-null   float64
 12  imdb_votes            2231 non-null   float64
 13  tmdb_popularity       2348 non-null   float64
 14  tmdb_score            2238 non-null   float64
dtypes: float64(5), int64(

In [3]:
netflix_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [4]:
# Concatentate datasets
combined_df = pd.concat([hulu_data, netflix_data])
combined_df.shape
combined_df.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       5840, 5841, 5842, 5843, 5844, 5845, 5846, 5847, 5848, 5849],
      dtype='int64', length=8248)

In [5]:
# Example of a value in the genres column with multiple words
combined_df['genres'][0]

# For the row with 'music' and 'comedy', we want to extract the first word 'music'
# and replace the entire value of this row's genres variable with 'music'

,genres
0,"['music', 'comedy']"
0,['documentation']


In [6]:
# Update the values in the genres column to be only one word

import ast

# Extracts the first genre from the genres 'list'
def extractFirstGenre(genreString):
    # Consider the genre missing if genreString is not a string and its length is less than or equal to 4
    if not isinstance(genreString, str) or len(genreString) <= 4:
        return pd.NA
    try:
        # Try to evaluate the genreString as a list
        genreList = ast.literal_eval(genreString)
        # Use the first value from this new list if it exists, otherwise consider the genre missing
        return genreList[0] if isinstance(genreList, list) and genreList else pd.NA
    except:
        return pd.NA

# Apply our defined function on the genres column
combined_df['genres'] = combined_df['genres'].apply(extractFirstGenre)
combined_df.reset_index(drop=True, inplace=True)

print("# Unique Genres: ", combined_df['genres'].nunique())
print("Unique Genres: ", combined_df['genres'].unique())

# Unique Genres:  18
Unique Genres:  ['music' 'war' 'comedy' 'drama' 'scifi' 'animation' 'family' 'fantasy'
 'action' 'documentation' 'crime' 'reality' 'western' 'thriller' 'romance'
 <NA> 'horror' 'history' 'sport']


In [7]:
# Post-transformation check. We expect each row for genres to be only one word.
combined_df['genres']

,genres
0,music
1,war
2,comedy
3,drama
4,comedy
...,...
8243,romance
8244,drama
8245,comedy
8246,<NA>


In [8]:
# Sanity check
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248 entries, 0 to 8247
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8248 non-null   object 
 1   title                 8247 non-null   object 
 2   type                  8248 non-null   object 
 3   description           8217 non-null   object 
 4   release_year          8248 non-null   int64  
 5   age_certification     4944 non-null   object 
 6   runtime               8248 non-null   int64  
 7   genres                8165 non-null   object 
 8   production_countries  8248 non-null   object 
 9   seasons               3436 non-null   float64
 10  imdb_id               7710 non-null   object 
 11  imdb_score            7600 non-null   float64
 12  imdb_votes            7583 non-null   float64
 13  tmdb_popularity       8107 non-null   float64
 14  tmdb_score            7777 non-null   float64
dtypes: float64(5), int64(

In [9]:
# Check for duplicate rows using the duplicated().sum() functions - returns number of duplicate rows
num_duplicates = combined_df.duplicated().sum()
print("Number of duplicate rows:", num_duplicates)

# Check if there are duplicate IDs
duplicates_exist = combined_df['id'].duplicated().any()
print("\nAre there duplicate IDs?:", duplicates_exist)

# Find the unique duplicate values
duplicate_values = combined_df[combined_df['id'].duplicated()]['id'].unique()
print("\nDuplicate ID values:", duplicate_values)

# Extract duplicate rows
duplicate_rows_df = combined_df[combined_df['id'].duplicated(keep=False)]
print("\nRows with Duplicate IDs:\n", duplicate_rows_df)

# Remove duplicates, keeping the row with the highest 'runtime'
deduplicated_df = (
    combined_df
    .sort_values(by='runtime', ascending=False)
    .drop_duplicates(subset=['id'], keep='first')
    .sort_values(by='id')
    .reset_index(drop=True)
)

duplicates_exist = deduplicated_df['id'].duplicated().any()
print("\nAre there duplicate IDs?:", duplicates_exist)

# Print the cleaned DataFrame without duplicate ids
print("\nDataFrame after removing duplicates while keeping the highest runtime:\n", deduplicated_df)

deduplicated_df.info()

Number of duplicate rows: 0

Are there duplicate IDs?: True

Duplicate ID values: ['tm180542' 'ts20678' 'ts20981' 'ts21223' 'ts20339' 'ts5145' 'ts34435'
 'tm192037' 'tm49178' 'ts25252' 'ts21034' 'tm191772' 'ts21693' 'ts20939'
 'ts12724' 'ts8308' 'ts21469' 'ts17156' 'ts22011' 'ts20604' 'ts11313'
 'ts20305' 'ts21511' 'tm133767' 'ts2230' 'ts21197' 'ts30313' 'ts12787'
 'ts35155' 'ts32186' 'tm148379' 'ts21740' 'ts20544' 'ts30288' 'ts25765'
 'ts21688' 'tm114476' 'ts20526' 'ts31593' 'tm53999' 'ts56589' 'tm170442'
 'ts32835' 'ts20433' 'ts20429' 'ts9246' 'tm174683' 'ts34151' 'ts191698'
 'tm183176' 'ts20682' 'ts20351' 'ts16045' 'ts32218' 'ts20014' 'ts22426'
 'ts521' 'ts37187' 'ts37143' 'ts37539' 'ts35201' 'ts35793' 'ts42014'
 'tm140391' 'ts52922' 'ts42169' 'ts38116' 'tm312676' 'tm366889' 'tm244364'
 'ts269185' 'tm414073' 'ts241248' 'ts84280' 'tm423436' 'tm363787'
 'ts216746' 'ts87466' 'ts191559' 'ts84633' 'tm823545' 'tm426638' 'ts87471'
 'tm463385' 'ts192297' 'ts252977' 'ts321614']

Rows with Du

In [10]:
# Combine the imdb_score and tmdb_score columns into one, averaged column.

# Transform tmdb_score to match imdb_score's scale (1-10)
deduplicated_df["tmdb_score_scaled"] = deduplicated_df["tmdb_score"] * (9/10) + 1

# Compute the final combined score (average)
deduplicated_df["average_score"] = deduplicated_df[["tmdb_score_scaled", "imdb_score"]].mean(axis=1, skipna=True)

# Print final number of rows after filtering
print("Final number of rows after removing rows with no scores:", len(deduplicated_df))

deduplicated_df.info()

Final number of rows after removing rows with no scores: 8161
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8161 non-null   object 
 1   title                 8160 non-null   object 
 2   type                  8161 non-null   object 
 3   description           8130 non-null   object 
 4   release_year          8161 non-null   int64  
 5   age_certification     4864 non-null   object 
 6   runtime               8161 non-null   int64  
 7   genres                8079 non-null   object 
 8   production_countries  8161 non-null   object 
 9   seasons               3370 non-null   float64
 10  imdb_id               7625 non-null   object 
 11  imdb_score            7516 non-null   float64
 12  imdb_votes            7499 non-null   float64
 13  tmdb_popularity       8020 non-null   float64
 14  tmdb_score

In [11]:
# Remove specified columns from the dataset using "drop()"
deduplicated_df = deduplicated_df.drop(columns=['description', 'id', 'imdb_id', 'imdb_score', 'tmdb_score', 'tmdb_score_scaled'])
deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 8160 non-null   object 
 1   type                  8161 non-null   object 
 2   release_year          8161 non-null   int64  
 3   age_certification     4864 non-null   object 
 4   runtime               8161 non-null   int64  
 5   genres                8079 non-null   object 
 6   production_countries  8161 non-null   object 
 7   seasons               3370 non-null   float64
 8   imdb_votes            7499 non-null   float64
 9   tmdb_popularity       8020 non-null   float64
 10  average_score         8039 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 701.5+ KB


In [12]:
print("# Unique Genres: ", deduplicated_df['genres'].nunique())
print("Unique Genres: ", deduplicated_df['genres'].unique())

# Unique Genres:  18
Unique Genres:  ['drama' 'scifi' 'action' 'music' 'comedy' 'animation' 'thriller'
 'documentation' 'romance' 'family' 'fantasy' 'horror' 'crime' 'sport'
 <NA> 'history' 'war' 'western' 'reality']


In [13]:
# Function to sort production_countries
def sort_countries(country_str):
    try:
        # Convert string to list
        country_list = ast.literal_eval(country_str)  # Converts string list to actual list
        # Sort list alphabetically
        country_list_sorted = sorted(country_list)
        # Convert back to string for consistency
        return str(country_list_sorted)
    except:
        return country_str  # Return as is if there's an issue

deduplicated_df["production_countries"] = deduplicated_df["production_countries"].apply(sort_countries)

deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 8160 non-null   object 
 1   type                  8161 non-null   object 
 2   release_year          8161 non-null   int64  
 3   age_certification     4864 non-null   object 
 4   runtime               8161 non-null   int64  
 5   genres                8079 non-null   object 
 6   production_countries  8161 non-null   object 
 7   seasons               3370 non-null   float64
 8   imdb_votes            7499 non-null   float64
 9   tmdb_popularity       8020 non-null   float64
 10  average_score         8039 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 701.5+ KB


In [14]:
print(deduplicated_df['production_countries'])
print("# Unique Production Coutnries: ", deduplicated_df['production_countries'].nunique())
print("Unique Production Countries: ", deduplicated_df['production_countries'].unique())

0                               ['CZ', 'DE']
1       ['CH', 'CZ', 'GB', 'HR', 'LU', 'RO']
2                                     ['GB']
3                                     ['US']
4                                     ['ID']
                        ...                 
8156                            ['CA', 'US']
8157                                  ['US']
8158                                  ['GB']
8159                                  ['US']
8160                                  ['SA']
Name: production_countries, Length: 8161, dtype: object
# Unique Production Coutnries:  483
Unique Production Countries:  ["['CZ', 'DE']" "['CH', 'CZ', 'GB', 'HR', 'LU', 'RO']" "['GB']" "['US']"
 "['ID']" "['PL']" "['IN', 'SU']" "['CN']" "['NG']" "['BR']"
 "['CA', 'NZ']" "['IN']" "['JP']" "['PH']" "['RU', 'US']" "['TW']"
 "['DE']" "['CA', 'US']" "['GB', 'IT']" "['KR', 'PL', 'US']"
 "['FR', 'US']" "['GB', 'US']" '[]' "['KR']" "['ES', 'FR', 'MX']" "['NL']"
 "['ES']" "['DK', 'IS']" "['ZA']" "['TR']" "['

In [15]:
# Remove any movies/shows without a title
deduplicated_df = deduplicated_df.dropna(subset=['title'])

In [16]:
# Showing the number of unique values in these columns;
# these can be converted to categorical variables
print(deduplicated_df['type'].nunique())
print(deduplicated_df['age_certification'].nunique())
print(deduplicated_df['genres'].nunique())

# Convert the above columns into categorical variables
deduplicated_df['type'] = deduplicated_df['type'].astype('category')
deduplicated_df['age_certification'] = deduplicated_df['age_certification'].astype('category')
deduplicated_df['genres'] = deduplicated_df['genres'].astype('category')

2
11
18


In [17]:
# Few more touches left

# Rename genres column to genre (more consistent with the stored value)
deduplicated_df.rename(columns={'genres': 'genre'}, inplace=True)

# Convert seasons and imdb_votes from float to int data type
deduplicated_df['seasons'] = deduplicated_df['seasons'].astype('Int64')
deduplicated_df['imdb_votes'] = deduplicated_df['imdb_votes'].astype('Int64')

# Reset the indexes of the rows.
deduplicated_df.reset_index(drop=True, inplace=True)

In [18]:
deduplicated_df.info()
final_df = deduplicated_df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8160 entries, 0 to 8159
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   title                 8160 non-null   object  
 1   type                  8160 non-null   category
 2   release_year          8160 non-null   int64   
 3   age_certification     4864 non-null   category
 4   runtime               8160 non-null   int64   
 5   genre                 8079 non-null   category
 6   production_countries  8160 non-null   object  
 7   seasons               3370 non-null   Int64   
 8   imdb_votes            7499 non-null   Int64   
 9   tmdb_popularity       8020 non-null   float64 
 10  average_score         8039 non-null   float64 
dtypes: Int64(2), category(3), float64(2), int64(2), object(2)
memory usage: 551.2+ KB


In [19]:
# Export the final merged/cleaned/wrangled dataset as csv
final_df.to_csv('merged.csv', index=False)